In [1]:
import os
import textwrap
import jsonlines
from google.cloud import dlp_v2
from collections import defaultdict

### General Evaluation

In [2]:
# load the inputs

notes = {}
with jsonlines.open('../Data/General/Input/notes-input.jsonl', 'r') as reader:
    for line in reader:
        notes[tuple(line['ID'])] = line['note']

In [3]:
# load the service

client = dlp_v2.DlpServiceClient().from_service_account_json(os.environ['GOOGLE_APPLICATION_CREDENTIALS'])

In [4]:
# make predictions

parent = 'projects/deidentification-370207'
inspect_config = {'info_types':[{'name':'PERSON_NAME'}]}
confidence = {'VERY_UNLIKELY':0.2, 'UNLIKELY':0.4, 'POSSIBLE':0.6, 'LIKELY':0.8, 'VERY_LIKELY':1.0}

outputs = defaultdict(dict)
for count, (ID, note) in enumerate(notes.items()):
    request = {'parent':parent, 'inspect_config':inspect_config, 'item':{'value':note}}
    response = client.inspect_content(request)
    
    for entity in response.result.findings:
        start, end = entity.location.byte_range.start, entity.location.byte_range.end
        name, likelihood = note[start:end], confidence[entity.likelihood.name]
        
        for title in ['Mr. ', 'Ms. ', 'Mrs. ', 'Dr. ', 'Dr.\n']:
            length = len(title)
            if name[:length] == title:
                name = name[length:]
                start += length
                break
        outputs[ID][(start, end)] = (name, likelihood)
        
    if count % 1000 == 0: print(f'Finish Processing Note {count}')

Finish Processing Note 0
Finish Processing Note 1000
Finish Processing Note 2000
Finish Processing Note 3000
Finish Processing Note 4000
Finish Processing Note 5000
Finish Processing Note 6000
Finish Processing Note 7000
Finish Processing Note 8000
Finish Processing Note 9000
Finish Processing Note 10000
Finish Processing Note 11000
Finish Processing Note 12000
Finish Processing Note 13000
Finish Processing Note 14000
Finish Processing Note 15000


In [5]:
# save the outputs

with jsonlines.open('../Data/General/Output/notes-Google.jsonl', 'w') as writer:
    writer.write_all([{'ID':list(ID), 'position':list(position), 'name':list(name)} for ID, preds in outputs.items() for position, name in preds.items()])

### Polysemy Evaluation

In [2]:
# load the inputs

notes = {}
with jsonlines.open('../Data/Polysemy/Input/polysemies-input.jsonl', 'r') as reader:
    for line in reader:
        notes[tuple(line['ID'])] = line['note']

In [3]:
# create a project with billing and DLP API enabled, set up a service account as DLP user and get credential, install DLP and set ENV variable
# load the service

client = dlp_v2.DlpServiceClient().from_service_account_json(os.environ['GOOGLE_APPLICATION_CREDENTIALS'])

In [4]:
# make predictions

parent = 'projects/deidentification-370207'
inspect_config = {'info_types':[{'name':'PERSON_NAME'}]}
confidence = {'VERY_UNLIKELY':0.2, 'UNLIKELY':0.4, 'POSSIBLE':0.6, 'LIKELY':0.8, 'VERY_LIKELY':1.0}

outputs = defaultdict(dict)
for count, (ID, note) in enumerate(notes.items()):
    request = {'parent':parent, 'inspect_config':inspect_config, 'item':{'value':note}}
    response = client.inspect_content(request)
    
    for entity in response.result.findings:
        start, end = entity.location.byte_range.start, entity.location.byte_range.end
        name, likelihood = note[start:end], confidence[entity.likelihood.name]
        
        for title in ['Mr. ', 'Ms. ', 'Mrs. ', 'Dr. ', 'Dr.\n']:
            length = len(title)
            if name[:length] == title:
                name = name[length:]
                start += length
                break
        outputs[ID][(start, end)] = (name, likelihood)
        
    if count % 100 == 0: print(f'Finish Processing Note {count}')

Finish Processing Note 0
Finish Processing Note 100
Finish Processing Note 200


In [5]:
# save the outputs

with jsonlines.open('../Data/Polysemy/Output/polysemies-Google.jsonl', 'w') as writer:
    writer.write_all([{'ID':list(ID), 'position':list(position), 'name':list(name)} for ID, preds in outputs.items() for position, name in preds.items()])